# __From raw twitter scrape json to clean fine tune babyfood jsonl__

## __snatch tweets from json__
reads a file that is a list of dictionaries where the target (tweet) is the value with a key named "text"

In [1]:
import json
 
# Opening JSON file
f = open('poetry_found.csv')
json_file = json.load(f)

text_file = []

for dictionary in json_file:
    text_file.append(dictionary["text"])
    

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

## __tweet cleaner__
if we decide hashtags, emojis or @mentions ae a part of contemporary expressions that belong into poetry we could comment out

In [63]:
import re
import emoji


def tweet_cleaner(tweet):
    # remove mentions
    mentions_free = re.sub("@\w+", "", tweet)
    # remove hashtags
    hashtag_free = re.sub("#\w+", "", mentions_free)
    # special shortcuts
    rt_free = re.sub("^RT", "", hashtag_free)
    # remove url 
    url_free = re.sub(r'http\S+', '', rt_free)
    # remove emojis
    emo_free = emoji.get_emoji_regexp().sub(u'', url_free)
    # remove white space
    stripped = emo_free.strip()
  
    return stripped

clean_tweets = [tweet_cleaner(x)for x in text_file]

## __unicode__
this removes stuff like \xa \n and so on

In [65]:
def unicode_maker(text):
    # creating a unicode string
    text_encode = text.encode(encoding="ascii", errors="ignore")
    text_decode = text_encode.decode()
    clean_text = " ".join([word for word in text_decode.split()])
    return clean_text

uni_tweets = [unicode_maker(x)for x in clean_tweets]
uni_tweets

[]

## __remove punctuation, numbers etc.__

In [66]:
import string

def number_punct(sentence):
    # remove numbers
    number_free = ''.join(word for word in sentence if not word.isdigit())
    # remove punctuation
    punctuation_free = "".join(
        [i for i in number_free if i not in string.punctuation])
    # lower case
    upper_free = punctuation_free.lower()
    
    return upper_free

number_free = [number_punct(x) for x in uni_tweets]
number_free

## __remove short tweets, duplicates, empty strings, excess whitespace__

In [68]:
## removes tweets with less than two words
more_than_two = [x for x in number_free if (len(x.split())>2)]

## removes duplicates
clean_set = set(more_than_two)
clean_set

# remove empty strings
no_empty_strings = [i for i in clean_set if i]
# remove outer whitespace 
stripped = [x.strip() for x in no_empty_strings]
# remove inner whitespace 
one_space =  [re.sub(' +', ' ', x) for x in stripped]
one_space

## __add whitespace in the beginning for openai's tokenization and a stop sign like END__

In [70]:
start_end = [f" {x}. END" for x in stripped] 
start_end

[]

## __Creates a DataFrame with a prompt and completion column__

In [71]:
import pandas as pd

# keyword = json_file[0]["segment"]
df_prompt_comp = pd.DataFrame({"prompt": "", "completion":start_end})
df_dub_free = df_prompt_comp.drop_duplicates()

#saves json babyfood
df_dub_free.to_json("....json", orient = "records", lines= True)

# __from txt file to babyfood__
saves some song examples in a txt file and get json with linebreaks and end bit (END) starting whitespace - songparts are devided by blank line (\n\n respectively) 

In [4]:
import re
import pandas as pd

In [6]:
# load txt file with song examples - examples are divided by whiteline
rap_file = list(open("raps.txt", "r"))
rap_list = "".join(x for x in rap_file)

# split examples by whiteline \n\n
chunks = re.split('\n\n', rap_list)

# replace \' by '
apo = [sub.replace("\'", "'") for sub in chunks]

# remove empty strings
no_empty_rap = [i for i in apo if i]

# put leading whitespace and END bit for better tokenization and GTP3 learning
space_rap = [f" {x} END" for x in no_empty_rap] 

# remove unicode
string_decode =  [x.encode("ascii", "ignore").decode() for x in space_rap] 

# make a pandas Dataframe
df_rap = pd.DataFrame({"prompt":"", "completion":string_decode})

# save into a GTP3 learnable jsonl with linebreaks and END bit to give as a example document 
df_rap.to_json("raps.json", orient = "records", lines= True)

# __make csv with poems to babyfood__

In [29]:
import pandas as pd
import re
from sklearn.model_selection import train_test_split

In [29]:
df_poetry = pd.read_csv("poetry_found.csv")
df_poetry2 = df_poetry["Poem"]

no_rr =  [re.sub(r'\r\r', '', x) for x in df_poetry2]
no_nnnn =  [re.sub(r'\n\n\n\n', '\n', x) for x in no_rr]
no_nnn =  [re.sub(r'\n\n\n', '\n', x) for x in no_nnnn]
no_nn =  [re.sub(r'\n\n', '\n', x) for x in no_nnn]
no_n =  [re.sub(r'\n \n', '\n', x) for x in no_nn]
stripped = [x.strip("\n") for x in no_n]
decode =  [x.encode("ascii", "ignore").decode() for x in stripped]
not_empty = [i for i in decode if i]
poem_onespace =  [re.sub(' +', ' ', x) for x in not_empty]

end_poem = [f" {x} END" for x in poem_onespace] 
df_poem = pd.DataFrame({"prompt":"", "completion":end_poem})
df_poem_singular = df_poem.drop_duplicates()

df_train, df_test = train_test_split(df_poem_singular, test_size=0.41)

In [143]:
df_train.to_json("poem_train.json", orient = "records", lines= True)
df_test.to_json("poem_test.json", orient = "records", lines= True)

In [140]:
df_poem_singular.shape

(13747, 2)

In [141]:
df_train.shape

(8110, 2)

In [142]:
df_test.shape

(5637, 2)